# core

> interect computer by LLM

In [ ]:
#| default_exp core
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def load_model(MODEL_PATH=None): 
    """ 加载模型 """
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

    if MODEL_PATH == None:
        MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"
    
    quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    config = AutoConfig.from_pretrained(MODEL_PATH)
    config.gradient_checkpointing = True
    config.do_sample = True
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        device_map = "auto",
        trust_remote_code = True,
        quantization_config=quantization_config,
        config=config
    )
    print(model)
    return model
    

In [ ]:
#| export
def resp_from_model(instruction,model=None):
    """ 获取模型回复"""
    if model==None:
        print('没有正常获取到模型！')
        return 
    import textwrap
    import transformers
    
    
    def make_inference(instruction):
        instruction_format = "[INST]{instruction}[/INST]"
        text = instruction_format.format(instruction=instruction)
        inputs = tokenizer(text, return_tensors="pt").to('cuda')
    
        outputs = model.generate(**inputs, max_new_tokens=2000)
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return output_text

    return make_inference(instruction)
 

In [ ]:
instruction = """
Write Python code to solve this problem in a brute force manner. You will only print the final result.

Find the largest two digit prime number which is also a prime number when the digits are reversed.

Reminder: You will write Python code. You do not care about the efficiency of the solution, you will only care about the correctness.
""".strip()
output_text = resp_from_model(instruction)
print(output_text)

In [ ]:
#| export
def code_extract(reply):
    import re
    def extract_code(reply):
        pattern = r"```python([\s\S]*?)```"
        matches = re.findall(pattern, reply)
        code = "\n\n".join(matches)
        if code:
            return code
        pattern = r"```([\s\S]*?)```"
        matches = re.findall(pattern, reply)
        code = "\n\n".join(matches)
        return code
    return extract_code(reply)

In [ ]:
code_extract('[INST]Write Python code to solve this problem in a brute force manner. You will only print the final result.\n\nFind the largest two digit prime number which is also a prime number when the digits are reversed.\n\nReminder: You will write Python code. You do not care about the efficiency of the solution, you will only care about the correctness.[/INST] Here is a brute force solution to find the largest two digit prime number which is also a prime number when the digits are reversed:\n\n```python\ndef is\\_prime(n):\nfor i in range(2, int(n**0.5) + 1):\nif n % i == 0:\nreturn False\nreturn True\n\nlargest\\_prime = -1\n')

''

In [ ]:
#| export
def code_exec(code, time_limit_seconds=20):
    import sys
    import io
    import contextlib
    import signal
    
    def run_python_code(code, time_limit_seconds=20):
        @contextlib.contextmanager
        def time_limit(seconds):
            def signal_handler(signum, frame):
                raise TimeoutError("Timed out!")
            signal.signal(signal.SIGALRM, signal_handler)
            signal.alarm(seconds)
            try:
                yield
            finally:
                signal.alarm(0)
    
        old_stdout = sys.stdout
        redirected_output = sys.stdout = io.StringIO()
    
        try:
            with time_limit(time_limit_seconds):
                exec(code)
        except TimeoutError as e:
            print("Code execution timed out")
        except Exception as e:
            print(f"Error: {str(e)}")
        finally:
            sys.stdout = old_stdout
    
        return redirected_output.getvalue()
    return run_python_code(code, time_limit_seconds=20)

In [ ]:
execution_output = code_exec("""
print(1+3)
""", time_limit_seconds=5)

print(execution_output)

Error: module 'signal' has no attribute 'SIGALRM'



In [ ]:
#| export
def solve_list_by_py(df_ori,problem = 'problem'):
    df = df_ori
    from collections import defaultdict

    instruction_template = """
    Write Python code to solve this problem in a brute force manner. You will only print the final result.
    
    {problem}
    
    Reminder:
    - You will write Python code.
    - You will only attempt to solve the problem with brute force.
    - You will only print the final result.
    """.strip()
    
    answers = []
    
    for problem in df[problem]:
        print(problem)
        responses = defaultdict(int)
        
        for _ in range(3):
            instruction = instruction_template.format(problem=problem)
            output_text = make_inference(instruction)
            print(f"{output_text=}")
            code = extract_code(output_text)
            print(f"{code=}")
            execution_output = run_python_code(code)
            print(f"{execution_output=}")
            try:
                result = int(execution_output)
                responses[result] += 1
            except ValueError:
                continue
            
        if len(responses) == 0:
            answer = 1
            answers.append(answer)  # random guess
        else:
            answer = sorted([(v,k) for k,v in responses.items()])[-1][-1]
            answers.append(answer%1000)  # most common guess
    
    df["answer"] = answers
    return df

In [ ]:
#| hide
import nbdev; 
nbdev.nbdev_export()